<a href="https://colab.research.google.com/github/aadhamashraf/ZC-SP24-DSAI-201/blob/main/Assignment2_Adham_Ashraf_202200953.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install the Pyterrier framework
!pip install python-terrier
# install the nltk modules
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.2/335.2 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.0 MB/s eta 0:00:00
  Preparing metadata (setup

In [15]:
#Import the necessary modules:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re # used to clean the data
from nltk.stem import *
from nltk.stem.porter import *
#from pyterrier import BatchRetrieve
import pyterrier as pt
import requests
from io import StringIO

if not pt.started(): pt.init()

In [10]:
stemmer = PorterStemmer()

In [ ]:
#to display the full text on the notebook without truncation
pd.set_option('display.max_colwidth', 150) ; nltk.download('stopwords') ; nltk.download('punkt')

In [5]:
def remove_stopwords(text):

    stop_words = stopwords.words('english')
    tokens = word_tokenize(text)
    filtered_tokens = [word.lower() for word in tokens if word.lower() not in stop_words]
    return ' '.join(filtered_tokens)


In [6]:
def Steem_text(text):

      tokens = word_tokenize(text)
      stemmed_tokens = [stemmer.stem(word) for word in tokens]
      return ' '.join(stemmed_tokens)


In [ ]:
import pandas as pd

url = "https://github.com/aadhamashraf/ZC-SP24-DSAI-201/raw/main/Assignments/Data/linkdin_Job_data.csv"

response = requests.get(url)

if response.status_code == 200:

    df = pd.read_csv(StringIO(response.text))
    df.dropna(inplace=True)
    df.drop_duplicates(inplace=True)

    df["docno"] = df["job_ID"].astype(str)
    df.dropna(subset=['job'], inplace=True)
    df.drop(columns=["job_ID"], inplace=True)

    df['processed_jobs'] = df['job'].apply(remove_stopwords)
    df['processed_jobs'] = df['job'].apply(Steem_text)

    df.set_index('docno', inplace=True)
    df.reset_index(inplace=True)
else:
    print("Failed to retrieve data from the URL:", url)


In [ ]:
indexer = pt.DFIndexer("./myFirstIndex", overwrite=True)
index_ref = indexer.index(df["processed_jobs"], df["docno"])

# **Result**

In [17]:
query="ai"

terrierindex = pt.IndexFactory.of(index_ref)
lex = terrierindex.getLexicon()
meta = terrierindex.getMetaIndex()
inv = terrierindex.getInvertedIndex()
le = lex.getLexiconEntry( query  )
job_ids=[] ; count =0
for posting in inv.getPostings( le ):
    docno = meta.getItem("docno", posting.getId())
    job_ids.append(docno) ; count+=1

In [ ]:
print(f"The number of {query} apperance is :",count,"\n With Existence in the following post's id :\n",job_ids)

In [22]:
filtered_df = df[df['docno'].isin(job_ids)]
filtered_df.reset_index(drop=True, inplace=True)

result=filtered_df.to_csv("Result.csv")